In [1]:

from torch.utils.data import DataLoader,TensorDataset
import torch.nn as nn
#import data_download
import random
import functions
#import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import torch
import polars as pl
from torch.nn import functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
dbname='ai_dataset'
dbuser='postgres'
dbpassword='parola'
dbport=5432
dbhost='127.0.0.1'
table_name='tabela'

In [3]:
tickers=["AAPL","MSFT",'SPY','XAUUSD.OANDA','BCO.ICMTRADER']
for i in range(len(tickers)):
    tickers[i]=tickers[i].replace('.','_')
net_functions=functions.Protected_execution

In [4]:
query=net_functions.create_query(tickers[0],table_name)
cursor,conn=net_functions.create_cursor(dbname,dbuser,dbpassword,dbport,dbhost)
data=pl.read_database(query=query,connection=conn)
#normalize with pandas

data=data.drop_nulls()
conn.close()
data

aapl_open,aapl_high,aapl_low,aapl_close,aapl_adj_close,aapl_volume
f64,f64,f64,f64,f64,f64
2.585,2.6696,2.5804,2.2722,2.2722,8.0723443e8
2.6832,2.7136,2.6607,2.2789,2.2789,6.196036e8
2.6725,2.675,2.6339,2.2609,2.2609,4.494224e8
2.6875,2.7393,2.6625,2.3193,2.3193,7.044576e8
2.7404,2.7571,2.705,2.3118,2.3118,6.7504077e8
…,…,…,…,…,…
195.18,195.41,192.97,193.6,193.6,3.71228e7
193.61,193.89,192.83,193.05,193.05,2.89193e7
192.49,193.5,191.09,193.15,193.15,4.80877e7


In [5]:
#get data from polars
#labels=data['labels']
train_data=data['aapl_high']
#make polars data into torch tensors
train_data=torch.tensor(train_data)

batch_size=32

In [6]:
#shorten the tensor
shortened_tensor=net_functions.tensor_shortner(train_data, batch_size)
#create batches of 32
image_tensor=net_functions.image_builder(shortened_tensor,batch_size=batch_size)
#create tensor
labels=net_functions.change(image_tensor, batch_size)

#cut the last image from data variable to adjust for change function
image_tensor=image_tensor[:-1]

In [7]:
#make labels and data into tensors
labels=torch.tensor(labels,dtype=torch.int32)
image_tensor=torch.tensor(image_tensor)

#move them to gpu
labels=labels.to(device)
image_tensor=image_tensor.to(device)


In [8]:
#split data with sklearn
train_dataset,test_dataset,train_labels,test_labels=train_test_split(image_tensor,labels,test_size=.1)

#create pythorch datasets
train_dataset=torch.utils.data.TensorDataset(train_dataset,train_labels)
test_dataset=torch.utils.data.TensorDataset(test_dataset,test_labels)
batchsize=32
train_loader=DataLoader(train_dataset,batch_size=batchsize,shuffle=True,drop_last=True)
#test_loader=DataLoader(test_data,batch_size=test_data.tensors[0].shape[0],drop_last=True)
test_loader=DataLoader(test_dataset,batch_size=batchsize,drop_last=True)
